# In this notebook we will be implementing Resnext50 and LSTM together

In [2]:
!pip install tensorflow

  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-1-py2.py3-none-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached keras-3.9.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorflow_io_gcs_filesystem-0.37.1-cp311-cp311-macosx_12_0_arm64.whl.metadata (14 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺ 246.2/252.6 MB 125.0 kB/s eta 0:00:5200:56
ERROR: Exception:
Traceback (most recent call last):
  File "/Users/kavyasmac/.local/share/virtualenvs/kavyasmac-wdQCnuT9/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/Users/kavyasmac/.loc

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers, models, applications
from tensorflow.keras.preprocessing.image import img_to_array

ModuleNotFoundError: No module named 'tensorflow'

In [9]:
os.getcwd()

'/Users/kavyasmac/Desktop/Time Pass/DeepFakeDetection/DeepfakeDetection/ModelFiles'

## Dataset Preparation

In [14]:
train_df = pd.read_csv('../CSV Files/train.csv')
print(len(train_df))
test_df = pd.read_csv('../CSV Files/test.csv')
print(len(test_df))

340
60


In [15]:
train_df.head()

,Index,Video Name,Tag
0,0,/Users/kavyasmac/Desktop/Time Pass/DeepFakeDet...,real
1,1,/Users/kavyasmac/Desktop/Time Pass/DeepFakeDet...,real
2,2,/Users/kavyasmac/Desktop/Time Pass/DeepFakeDet...,real
3,3,/Users/kavyasmac/Desktop/Time Pass/DeepFakeDet...,real
4,4,/Users/kavyasmac/Desktop/Time Pass/DeepFakeDet...,real


## Feed Videos to a network

In [23]:
#This function takes the path of a video, resizes its frames for resnext50 and returns a list of frames.
IMG_SIZE = 224
SEQ_LENGTH = 10  # Number of frames per video
NUM_CLASSES = 2 
def load_video(path, max_frames=SEQ_LENGTH, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, resize)
            frames.append(frame)
            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

## Feature Extraction

In [20]:
df = pd.read_csv("../CSV Files/train.csv")
df.head()

,Index,Video Name,Tag
0,0,/Users/kavyasmac/Desktop/Time Pass/DeepFakeDet...,real
1,1,/Users/kavyasmac/Desktop/Time Pass/DeepFakeDet...,real
2,2,/Users/kavyasmac/Desktop/Time Pass/DeepFakeDet...,real
3,3,/Users/kavyasmac/Desktop/Time Pass/DeepFakeDet...,real
4,4,/Users/kavyasmac/Desktop/Time Pass/DeepFakeDet...,real


In [25]:
# Preparing dataset
video_paths = df["Video Name"].values
labels = df["Tag"].map({"real": 0, "fake": 1}).values

In [27]:
test_video_paths = test_df["Video Name"].values
test_video_labels = test_df["Tag"].map({"real": 0, "fake": 1}).values

In [28]:
# Load video frames
X_train = np.array([load_video(path) for path in video_paths])
y_train = np.array(labels)
X_test = np.array([load_video(path) for path in test_video_paths])
y_test = np.array(test_video_labels)


In [50]:
# Define Model
class DeepfakeModel(tf.keras.Model):
    def __init__(self, num_classes, hidden_dim=2048, bidirectional=False):
        super(DeepfakeModel, self).__init__()
        base_model = applications.ResNeXt50(include_top=False, weights='imagenet')
        self.model = models.Sequential(base_model.layers[:-2])  # Feature extractor
        self.avgpool = layers.GlobalAveragePooling2D()
        self.lstm = layers.Bidirectional(layers.LSTM(hidden_dim, return_sequences=True)) if bidirectional else layers.LSTM(hidden_dim, return_sequences=True)
        self.dropout = layers.Dropout(0.4)
        self.fc = layers.Dense(num_classes, activation="softmax")

    def call(self, x):
        batch_size, seq_length, h, w, c = tf.shape(x)
        x = tf.reshape(x, (batch_size * seq_length, h, w, c))
        fmap = self.model(x)  # Feature Extraction
        x = self.avgpool(fmap)
        x = tf.reshape(x, (batch_size, seq_length, -1))
        x_lstm = self.lstm(x)
        x_mean = tf.reduce_mean(x_lstm, axis=1)
        return fmap, self.dropout(self.fc(x_mean))

In [51]:
model = DeepfakeModel(NUM_CLASSES)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

AttributeError: module 'keras.api.applications' has no attribute 'ResNeXt50'

In [52]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=8)

Epoch 1/10


AttributeError: Exception encountered when calling DeepfakeModel.call().

[1m'DeepfakeModel' object has no attribute 'model'[0m

Arguments received by DeepfakeModel.call():
  • x=tf.Tensor(shape=(None, 10, 224, 224, 3), dtype=uint8)

In [3]:
class Model(tf.keras.Model):
    def __init__(self, num_classes, latent_dim=2048, lstm_layers=1, hidden_dim=2048, bidirectional=False):
        super(Model, self).__init__()
        base_model = applications.ResNeXt50(include_top=False, weights='imagenet')
        self.model = models.Sequential(base_model.layers[:-2])
        self.avgpool = layers.GlobalAveragePooling2D()
        self.lstm = layers.Bidirectional(layers.LSTM(hidden_dim, return_sequences=True)) if bidirectional else layers.LSTM(hidden_dim, return_sequences=True)
        self.relu = layers.LeakyReLU()
        self.dp = layers.Dropout(0.4)
        self.linear1 = layers.Dense(num_classes)

    def call(self, x):
        batch_size, seq_length, c, h, w = tf.shape(x)
        x = tf.reshape(x, (batch_size * seq_length, h, w, c))
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = tf.reshape(x, (batch_size, seq_length, -1))
        x_lstm = self.lstm(x)
        x_mean = tf.reduce_mean(x_lstm, axis=1)
        return fmap, self.dp(self.linear1(x_mean))

In [ ]:
# Function to extract frames from a video
def extract_frames(video_path, frame_size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read() 
        if not ret:
            break
        frame = cv2.resize(frame, frame_size)
        frame = img_to_array(frame)
        frames.append(frame)
    cap.release()
    return np.array(frames)

In [5]:
# Function to create a dataset from the video folders
def create_dataset(video_dir, frame_size=(224, 224), seq_length=10):
    classes = ['real', 'fake']
    data = []
    labels = []
    for class_idx, class_name in enumerate(classes):
        class_dir = os.path.join(video_dir, class_name)
        for video_name in os.listdir(class_dir):
            video_path = os.path.join(class_dir, video_name)
            frames = extract_frames(video_path, frame_size)
            if len(frames) >= seq_length:
                for i in range(0, len(frames) - seq_length + 1, seq_length):
                    data.append(frames[i:i + seq_length])
                    labels.append(class_idx)
    data = np.array(data)
    labels = np.array(labels)
    return data, labels

In [ ]:
# Paths to the video folders
train_dir = '/Users/kavyasmac/Desktop/Time Pass/DeepFakeDetection/Data/ModelData/Train'

# Create the dataset
frame_size = (224, 224)
seq_length = 10
X_train, y_train = create_dataset(train_dir, frame_size, seq_length)

# Convert to TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(8)

# Define the model
num_classes = 2
model = Model(num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_dataset, epochs=10)